## 1. Import Dependencies

In [3]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

## 2. Building an Environment

In [12]:
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box

class Hot_or_Cold_Env(Env):
    def __init__(self):
        # Actions we can take: left, right, forward, backward
        self.action_space = Discrete(4)
        
        # Observation space: 100x100 grid
        self.observation_space = Box(low=0, high=99, shape=(2,), dtype=np.int32)
       
        # Randomly initialize the agent's starting position within the 100x100 grid
        self.state = np.array([np.random.randint(0, 100), np.random.randint(0, 100)])
        
        # Set the target position randomly
        self.target = np.array([np.random.randint(0, 100), np.random.randint(0, 100)])
        
        # Ensure the target is not in the same position as the agent
        while np.array_equal(self.state, self.target):
            self.target = np.array([np.random.randint(0, 100), np.random.randint(0, 100)])

        # Set the period duration (in seconds)
        self.period_duration = 60 
        
        # Initialize step count
        self.steps_taken = 0
        
        # Initialize previous distance
        self.previous_distance_to_target = np.inf
        
    def step(self, action):
        # Apply action
        if action == 0:  # backward
            self.state[1] = max(0, self.state[1] - 1)  # y ekseninde -1, alt sınır
        elif action == 1:  # right
            self.state[0] = min(99, self.state[0] + 1)  # x ekseninde +1, üst sınır
        elif action == 2:  # forward
            self.state[1] = min(99, self.state[1] + 1)  # y ekseninde +1, üst sınır
        elif action == 3:  # left
            self.state[0] = max(0, self.state[0] - 1)  # x ekseninde -1, alt sınır
        
        # Reduce game length by 1 second
        self.period_duration -= 1 
        
        # Increment step count
        self.steps_taken += 1
        
        # Calculate the reward
        reward = -0.1  # Small negative reward for each step taken
        
        # Calculate distance to the target
        distance_to_target = np.linalg.norm(self.state - self.target)
        
        # Check if the agent reached the target
        if np.array_equal(self.state, self.target):
            reward += 100  # Large positive reward for reaching the target
            done = True  # End the episode
        else:
            done = False  # Not done yet
            
            # Update previous distance
            if self.steps_taken > 1:  # Skip this check for the first step
                if distance_to_target < self.previous_distance_to_target:
                    reward += 1  # Positive reward for getting closer
                else:
                    reward -= 0.5  # Negative reward for getting further away
            
        # Update previous distance
        self.previous_distance_to_target = distance_to_target  
        
        # Check if the game is done
        if self.period_duration <= 0:
            done = True
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def reset(self):
        # Reset the environment
        self.state = np.array([np.random.randint(0, 100), np.random.randint(0, 100)])
        self.target = np.array([np.random.randint(0, 100), np.random.randint(0, 100)])  # New target
        
        # Ensure the target is not in the same position as the agent
        while np.array_equal(self.state, self.target):
            self.target = np.array([np.random.randint(0, 100), np.random.randint(0, 100)])

        self.period_duration = 60  # Reset period duration
        self.steps_taken = 0  # Reset step count
        self.previous_distance_to_target = np.inf  # Initialize previous distance
        return self.state

## 3.Test Environment

In [ ]:
# The Hot_or_Cold_Env class definition should be placed here.

# Test function for the Hot or Cold environment
def test_hot_or_cold_env():
    env = Hot_or_Cold_Env()  # Create an instance of the environment
    state = env.reset()  # Reset the environment to get the initial state
    
    print("Initial State:", state)
    print("Target:", env.target)

    total_reward = 0  # Variable to track the total reward
    for step in range(10):  # Perform a maximum of 10 steps
        action = np.random.choice(4)  # Randomly select an action
        next_state, reward, done, info = env.step(action)  # Take the action and get the next state and reward

        # Update the total reward
        total_reward += reward

        # Print the current state and reward information
        print(f"Step {step + 1}:")
        print(f"  Action: {action}")
        print(f"  New State: {next_state}")
        print(f"  Reward: {reward}")
        print(f"  Total Reward: {total_reward}")
        print(f"  Distance to Target: {np.linalg.norm(next_state - env.target)}")
        
        if done:  # Check if the episode is done
            print("Agent reached the target or time has run out.")
            break  # Exit the loop if done

# Run the test
test_hot_or_cold_env()

## 4.Model Training with PPO / MlPolicy

In [13]:
log_path = os.path.join('Training', 'Logs')

In [14]:
# Create the environment
env = Hot_or_Cold_Env()

In [27]:
# Create the PPO model
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/can/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [16]:
# Train the model
model.learn(total_timesteps=1000000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 58.9     |
|    ep_rew_mean     | 10.9     |
| time/              |          |
|    fps             | 7768     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 59.5        |
|    ep_rew_mean          | 10.6        |
| time/                   |             |
|    fps                  | 5661        |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.017111693 |
|    clip_fraction        | 0.267       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_variance   | -0.00584    |
|    learning_rate        | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 14.3        |
| time/                   |             |
|    fps                  | 4640        |
|    iterations           | 11          |
|    time_elapsed         | 4           |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.010049952 |
|    clip_fraction        | 0.0548      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.23       |
|    explained_variance   | 0.314       |
|    learning_rate        | 0.0003      |
|    loss                 | 6.49        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00538    |
|    value_loss           | 15.7        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 18.2         |
| time/                   |              |
|    fps                  | 4563         |
|    iterations           | 21           |
|    time_elapsed         | 9            |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0058709756 |
|    clip_fraction        | 0.0646       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.21        |
|    explained_variance   | 0.675        |
|    learning_rate        | 0.0003       |
|    loss                 | 4.81         |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.00408     |
|    value_loss           | 13.4         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 18.1         |
| time/                   |              |
|    fps                  | 4515         |
|    iterations           | 31           |
|    time_elapsed         | 14           |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0064889845 |
|    clip_fraction        | 0.069        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.16        |
|    explained_variance   | 0.625        |
|    learning_rate        | 0.0003       |
|    loss                 | 3.4          |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.00579     |
|    value_loss           | 12.2         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 59.9        |
|    ep_rew_mean          | 17.7        |
| time/                   |             |
|    fps                  | 4474        |
|    iterations           | 41          |
|    time_elapsed         | 18          |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.008330135 |
|    clip_fraction        | 0.148       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.16       |
|    explained_variance   | 0.65        |
|    learning_rate        | 0.0003      |
|    loss                 | 6.23        |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.00281    |
|    value_loss           | 12.4        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 59.9  

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 59.8       |
|    ep_rew_mean          | 20.9       |
| time/                   |            |
|    fps                  | 4028       |
|    iterations           | 441        |
|    time_elapsed         | 224        |
|    total_timesteps      | 903168     |
| train/                  |            |
|    approx_kl            | 0.01796164 |
|    clip_fraction        | 0.12       |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.749     |
|    explained_variance   | 0.542      |
|    learning_rate        | 0.0003     |
|    loss                 | 8.71       |
|    n_updates            | 4400       |
|    policy_gradient_loss | -0.00342   |
|    value_loss           | 17.2       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 59.8        |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 19.3        |
| time/                   |             |
|    fps                  | 4009        |
|    iterations           | 451         |
|    time_elapsed         | 230         |
|    total_timesteps      | 923648      |
| train/                  |             |
|    approx_kl            | 0.009155864 |
|    clip_fraction        | 0.0913      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.807      |
|    explained_variance   | 0.347       |
|    learning_rate        | 0.0003      |
|    loss                 | 10.6        |
|    n_updates            | 4500        |
|    policy_gradient_loss | -0.00453    |
|    value_loss           | 24.2        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 59.5  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 59.6         |
|    ep_rew_mean          | 19.3         |
| time/                   |              |
|    fps                  | 3985         |
|    iterations           | 461          |
|    time_elapsed         | 236          |
|    total_timesteps      | 944128       |
| train/                  |              |
|    approx_kl            | 0.0059584053 |
|    clip_fraction        | 0.0752       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.712       |
|    explained_variance   | 0.637        |
|    learning_rate        | 0.0003       |
|    loss                 | 6.97         |
|    n_updates            | 4600         |
|    policy_gradient_loss | -0.00252     |
|    value_loss           | 21.3         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 18.7        |
| time/                   |             |
|    fps                  | 3963        |
|    iterations           | 471         |
|    time_elapsed         | 243         |
|    total_timesteps      | 964608      |
| train/                  |             |
|    approx_kl            | 0.011978266 |
|    clip_fraction        | 0.186       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.8        |
|    explained_variance   | 0.544       |
|    learning_rate        | 0.0003      |
|    loss                 | 4.47        |
|    n_updates            | 4700        |
|    policy_gradient_loss | -0.0019     |
|    value_loss           | 14          |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 59.6

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 18.2         |
| time/                   |              |
|    fps                  | 3941         |
|    iterations           | 481          |
|    time_elapsed         | 249          |
|    total_timesteps      | 985088       |
| train/                  |              |
|    approx_kl            | 0.0056273337 |
|    clip_fraction        | 0.0869       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.752       |
|    explained_variance   | 0.403        |
|    learning_rate        | 0.0003       |
|    loss                 | 11           |
|    n_updates            | 4800         |
|    policy_gradient_loss | 0.000103     |
|    value_loss           | 20.9         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

In [22]:
evaluate_policy(model, env, n_eval_episodes=1000, render=True)

(19.06729859870672, 23.313152556109294)

In [28]:
model.save(os.path.join('Saved Models', 'PPO_1MT'))

/Users/can/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
